In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.__version__

'1.6.0'

In [3]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-
from __future__ import division, print_function, unicode_literals
import tensorflow as tf
from time import time
from keras.utils import np_utils#one hot
import os
from skimage import data
from skimage import transform
import skimage
import numpy as np

# Download the dataset


def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                  if os.path.isdir(os.path.join(data_directory,d))]
    labels=[]
    images=[]
    for d in directories:
        label_directory = os.path.join(data_directory,d)
        file_names = [os.path.join(label_directory,f)
                     for f in os.listdir(label_directory)
                     if f.endswith('.jpg')]
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(str(d))
    return images, labels






Using TensorFlow backend.


In [4]:
ROOT_PATH = '/home/tang/targa/BigSize1/'
train_data_directory = os.path.join(ROOT_PATH, "train")
validation_data_directory = os.path.join(ROOT_PATH, "val")
test_data_directory = os.path.join(ROOT_PATH, "test")
train_images,train_labels = load_data(train_data_directory)
test_images,test_labels = load_data(test_data_directory)
validation_images,validation_labels = load_data(validation_data_directory)

In [5]:
#encoding trY teY
from numpy import array
#from numpy import argmax
#from keras.utils import to_categorical
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example

#train_labels = np_utils.to_categorical(train_labels,num_classes = None)
#test_labels = np_utils.to_categorical(test_labels,num_classes = None)
trL,teL,valL=array(train_labels),array(test_labels),array(validation_labels)
label_encoder = LabelEncoder()

In [6]:
integer_encoded_trL = label_encoder.fit_transform(trL)
integer_encoded_teL = label_encoder.fit_transform(teL)
integer_encoded_valL = label_encoder.fit_transform(valL)

In [7]:
trainLabels = np_utils.to_categorical(integer_encoded_trL,num_classes = 32)
testLabels = np_utils.to_categorical(integer_encoded_teL,num_classes = 32)
validationLabels = np_utils.to_categorical(integer_encoded_valL,num_classes = 32)

In [8]:
########convert rgb images into black and white
#convert images into size of vector
from skimage.color import rgb2gray

trainImages = rgb2gray(np.array(train_images))
testImages = rgb2gray(np.array(test_images))
valImages = rgb2gray(np.array(validation_images))
train_flat_images = np.reshape(trainImages,(len(trainImages),160*90))
test_flat_images = np.reshape(testImages,(len(testImages),160*90))
val_flat_images = np.reshape(valImages,(len(valImages),160*90))

In [9]:
valImages.shape

(37312, 160, 90)

In [10]:
train_flat_images[0].shape

(14400,)

In [11]:
print('train images shape:\t\t{}'.format(train_flat_images.shape))

print('test images shape:\t\t{}'.format(test_flat_images.shape))

print('val images shape:\t\t{}'.format(val_flat_images.shape))

train images shape:		(37312, 14400)
test images shape:		(37312, 14400)
val images shape:		(37312, 14400)


In [12]:
integer_encoded_trL[0]

18

In [13]:
validationLabels[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [14]:
print('Size of:')
print('- Training-set:\t\t{}'.format(len(trainLabels)))
print('- Test-set:\t\t{}'.format(len(testLabels)))
print('- Validation-set:\t{}'.format(len(validation_labels)))

Size of:
- Training-set:		37312
- Test-set:		37312
- Validation-set:	37312


In [15]:
batch_size = 100
n_batch = len(trainLabels)//batch_size
lr = tf.Variable(0.001,dtype = tf.float32)

In [17]:
#命名空间
with tf.name_scope('input'):
    #定义两个placeholder 
    #把批次的图片传进去，开始是100张一次的传送，直到最后
    x = tf.placeholder(tf.float32,[None,14400],name='x-input')
    y = tf.placeholder(tf.float32,[None,32],name='y-input')
#定义一个dropout的值
keep_prob = tf.placeholder(tf.float32)
#第一层
W1 = tf.Variable(tf.truncated_normal([14400,1000],stddev=0.1))#改成截断正太分布
b1 = tf.Variable(tf.zeros([1000])+0.1)
output1 = tf.matmul(x,W1) + b1
L1 = tf.nn.tanh(output1)
#L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
L1_drop = tf.nn.dropout(L1,keep_prob)

#第二层
W2 = tf.Variable(tf.truncated_normal([1000,2000],stddev=0.1))
b2 = tf.Variable(tf.zeros([2000])+0.2)
output2 = tf.matmul(L1_drop,W2)+b2
L2 = tf.nn.tanh(output2)
#L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
L2_drop = tf.nn.dropout(L2,keep_prob)



#第三层
W3 = tf.Variable(tf.truncated_normal([2000,1000],stddev=0.1))
b3 = tf.Variable(tf.zeros([1000])+0.1)
output3 = tf.matmul(L2_drop,W3)+b3
L3 = tf.nn.tanh(output3)
#L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
L3_drop = tf.nn.dropout(L3,keep_prob)

# 第四层
W4 = tf.Variable(tf.truncated_normal([1000,32],stddev=0.1))
b4 = tf.Variable(tf.zeros([32])+0.1)
output4 = tf.matmul(L3_drop,W4)+ b4


#预测分类
prediction = tf.nn.softmax(output4)

#定义一个损失函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits = prediction))#联合交叉熵
#loss = tf.reduce_mean(tf.square(y-prediction))
#定义一个方法来收敛神经网络，使用梯度下降法,学习率=0.1
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
#train_step = tf.train.AdamOptimizer(0.1).minimize(loss)
#初始化
init = tf.global_variables_initializer()

#定义一个求准确率的方法
#如果有两个相同的就会返回TRUE，不相同就返回false，然后就依次存入correct_prediction
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中的最大值

#求准确率
#首先把bool值转化成32位的浮点数，然后求平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [18]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [19]:
with tf.Session() as sess:
    sess.run(init)
    #路径和文件名
    write = tf.summary.FileWriter('logs/fourLayerFullyConnectedTarga',sess.graph)
    for epoch in range(200):#把所有的图片训练20次
        #一批一批的迭代图片，一次迭代100张，一共运行n_batch次
        for batch in range(n_batch):
            #每次传入一百张图片
            batch_xs,batch_ys = next_batch(batch_size,train_flat_images,trainLabels)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
        
        acc = sess.run(accuracy,feed_dict={x:test_flat_images,y:testLabels,keep_prob:1.0})
        train_acc = sess.run(accuracy,feed_dict={x:train_flat_images,y:trainLabels,keep_prob:1.0})
        print('epoch=' + str(epoch) + 'test accuracy=' + str(acc)+'train accuracy='+ str(train_acc))

epoch=0test accuracy=0.5171795train accuracy=0.53192
epoch=1test accuracy=0.5598467train accuracy=0.58434284
epoch=2test accuracy=0.57670456train accuracy=0.6069629
epoch=3test accuracy=0.5854685train accuracy=0.62443715
epoch=4test accuracy=0.5978774train accuracy=0.6430639
epoch=5test accuracy=0.60278195train accuracy=0.65474916
epoch=6test accuracy=0.6079009train accuracy=0.6649603
epoch=7test accuracy=0.61095625train accuracy=0.6740191
epoch=8test accuracy=0.6138508train accuracy=0.68149656
epoch=9test accuracy=0.61816573train accuracy=0.6899121
epoch=10test accuracy=0.618943train accuracy=0.69505787
epoch=11test accuracy=0.6180853train accuracy=0.697068
epoch=12test accuracy=0.6216767train accuracy=0.70516187
epoch=13test accuracy=0.6252948train accuracy=0.7110313
epoch=14test accuracy=0.6257504train accuracy=0.7146226
epoch=15test accuracy=0.6300654train accuracy=0.72180533
epoch=16test accuracy=0.6304406train accuracy=0.7248606
epoch=17test accuracy=0.6481561train accuracy=0.744

epoch=144test accuracy=0.72121567train accuracy=0.89185786
epoch=145test accuracy=0.7215641train accuracy=0.89199185
epoch=146test accuracy=0.7221269train accuracy=0.8918846
epoch=147test accuracy=0.7190984train accuracy=0.8905446
epoch=148test accuracy=0.7219125train accuracy=0.89266187
epoch=149test accuracy=0.72274333train accuracy=0.89271545
epoch=150test accuracy=0.7223681train accuracy=0.8927691
epoch=151test accuracy=0.72210014train accuracy=0.8927691
epoch=152test accuracy=0.7196344train accuracy=0.89215267
epoch=153test accuracy=0.72282374train accuracy=0.8931711
epoch=154test accuracy=0.7213765train accuracy=0.8931443
epoch=155test accuracy=0.7214837train accuracy=0.8928227
epoch=156test accuracy=0.7198489train accuracy=0.8929031
epoch=157test accuracy=0.7215641train accuracy=0.8932783
epoch=158test accuracy=0.7222073train accuracy=0.8937875
epoch=159test accuracy=0.72306496train accuracy=0.8941627
epoch=160test accuracy=0.7231722train accuracy=0.89461833
epoch=161test accura

In [ ]:
#change learning rate?